In [1]:
import os
import io
from zipfile import ZipFile

In [2]:
import shutil
import csv

In [3]:
import re

In [4]:
import xapian

ImportError: /anaconda/envs/py35/lib/python3.5/site-packages/xapian/_xapian.cpython-35m-x86_64-linux-gnu.so: undefined symbol: _ZN6Xapian12TermIterator7skip_toERKSs

In [5]:
import pandas as pd
import numpy as np

In [6]:
pd.options.display.max_colwidth = 100

In [7]:
dbpath = "full_index"

# Build Index

In [6]:
zf = ZipFile("../wiki-pages-text.zip")

In [7]:
files = [item.filename for item in zf.filelist]
len(files)

110

In [8]:
def read_shard(zf, path):
    items = []
    fp = zf.open(path, mode='r')
    tfp = io.TextIOWrapper(fp)
    for line in tfp.readlines():
        match = re.match("(\w+)\s(\d+)\s(.*)\n", line)
        if match:
            items.append(match.groups())
    fp.close()
    tfp.close()
    return items

In [10]:
def read_shard_as_df(zf, path):
    items = read_shard(zf, path)
    raw_df = pd.DataFrame(data=items, columns=['doc_id', 'sentence', 'text'])
    func = lambda x: " ".join(x)
    return raw_df.groupby('doc_id')['text'].agg(func)

In [69]:
%%time
# Create or open the database we're going to be writing to.
db = xapian.WritableDatabase(dbpath, xapian.DB_CREATE_OR_OPEN)

# Set up a TermGenerator that we'll use in indexing.
termgenerator = xapian.TermGenerator()
termgenerator.set_stemmer(xapian.Stem("en"))

for path in sorted(files[1:]):
    print(path)
    documents_df = read_shard_as_df(zf, path)
    for doc_id, text in documents_df.items():
        # We make a document and tell the term generator to use this.
        doc = xapian.Document()
        termgenerator.set_document(doc)

        # Index each field with a suitable prefix.
        termgenerator.index_text(doc_id, 1, 'S')
        #termgenerator.index_text(description, 1, 'XD')

        # Index fields without prefixes for general search.
        termgenerator.index_text(text)
        termgenerator.increase_termpos()
        #termgenerator.index_text(description)

        # We use the identifier to ensure each object ends up in the
        # database only once no matter how many times we run the
        # indexer.
        idterm = u"Q" + doc_id
        doc.add_boolean_term(idterm)
        db.replace_document(idterm, doc)
db.commit()
db.close()

wiki-pages-text/wiki-009.txt
wiki-pages-text/wiki-021.txt
wiki-pages-text/wiki-035.txt
wiki-pages-text/wiki-034.txt
wiki-pages-text/wiki-020.txt
wiki-pages-text/wiki-008.txt
wiki-pages-text/wiki-036.txt
wiki-pages-text/wiki-022.txt
wiki-pages-text/wiki-023.txt
wiki-pages-text/wiki-037.txt
wiki-pages-text/wiki-033.txt
wiki-pages-text/wiki-027.txt
wiki-pages-text/wiki-026.txt
wiki-pages-text/wiki-032.txt
wiki-pages-text/wiki-024.txt
wiki-pages-text/wiki-030.txt
wiki-pages-text/wiki-018.txt
wiki-pages-text/wiki-019.txt
wiki-pages-text/wiki-031.txt
wiki-pages-text/wiki-025.txt
wiki-pages-text/wiki-042.txt
wiki-pages-text/wiki-056.txt
wiki-pages-text/wiki-081.txt
wiki-pages-text/wiki-095.txt
wiki-pages-text/wiki-094.txt
wiki-pages-text/wiki-080.txt
wiki-pages-text/wiki-057.txt
wiki-pages-text/wiki-043.txt
wiki-pages-text/wiki-069.txt
wiki-pages-text/wiki-055.txt
wiki-pages-text/wiki-041.txt
wiki-pages-text/wiki-096.txt
wiki-pages-text/wiki-082.txt
wiki-pages-text/wiki-109.txt
wiki-pages-tex

In [11]:
!xapian-delve $dbpath

UUID = b940719a-3fb5-4d57-8b01-3186790b1ade
number of documents = 3833466
average document length = 170.939
document length lower bound = 1
document length upper bound = 31154
highest document id ever used = 3833466
has positional information = true
revision = 392
currently open for writing = false


In [16]:
!xapian-delve -r 1833129 -d $dbpath

Data for record #1833129:

Term List for record #1833129: 17 2004 2007 3 QKim_Hyde Skim_hyde ZSkim_hyd Za Zand Zappear Zaustralian Zaway Zby Zcharact Zchris Zdepart Zfebruari Zfiction Zfirst Zfrom Zhe Zhemsworth Zhis Zhome Zhyde Zjonathan Zjuli Zkim Zkimber Zmade Zon Zopera Zplay Zscreen Zsoap Zthe Zwas a and appearance australian away by character chris departed february fictional first from he hemsworth his home hyde jonathan july kim kimberly made on opera played screen soap the was
